In [2]:
import os

# pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()

TMDB_API_KEY = os.getenv('TMDB_API_KEY')
TMDB_API_TOKEN = os.getenv('TMDB_API_TOKEN')

if TMDB_API_KEY and TMDB_API_TOKEN:
    print('✅API KEY and TOKEN are set!')
else:
    print('❌API KEY and TOKEN 404')

✅API KEY and TOKEN are set!


In [7]:
import asyncio
import aiohttp
import pandas as pd
import os
import time
import nest_asyncio

# 주피터 노트북 환경에서 비동기 충돌을 해결합니다.
nest_asyncio.apply()

# TMDB API 토큰을 환경 변수에서 불러옵니다.
# 'TMDB_API_TOKEN'이라는 환경 변수에 여러분의 API 토큰을 설정해야 합니다.
TMDB_API_TOKEN = os.environ.get('TMDB_API_TOKEN')

# API 요청에 필요한 헤더
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {TMDB_API_TOKEN}"
}

# 모든 영화 데이터를 담을 빈 리스트
all_movie_data = []

# 동시 API 요청 수를 제한합니다.
# 이 값을 너무 크게 설정하면 'too many file descriptors' 오류가 발생할 수 있습니다.
# 50으로 줄이면 안정성이 높아집니다.
CONCURRENT_REQUESTS = 50
semaphore = asyncio.Semaphore(CONCURRENT_REQUESTS)

# 재시도 횟수
MAX_RETRIES = 5

async def fetch_movie_details(session, movie_id):
    """
    비동기적으로 단일 영화의 상세 정보를 가져오는 함수 (재시도 로직 포함)
    """
    # 세마포어를 사용하여 동시 요청 수를 제어합니다.
    async with semaphore:
        url_details = f"https://api.themoviedb.org/3/movie/{movie_id}"
        
        for retry in range(MAX_RETRIES):
            try:
                async with session.get(url_details, headers=headers) as detail_response:
                    if detail_response.status == 200:
                        details = await detail_response.json()
                        movie_data = {
                            'id': details.get('id'),
                            'title': details.get('original_title'), # 'title' 대신 'original_title' 사용
                            'release_date': details.get('release_date'),
                            'budget': details.get('budget'),
                            'revenue': details.get('revenue'),
                            'runtime': details.get('runtime'),
                            'vote_average': details.get('vote_average')
                        }
                        return movie_data
                    elif detail_response.status == 429:
                        print(f"상세 정보 요청 실패: 429 (ID: {movie_id}), API 제한 초과! 60초 대기 후 재시도...")
                        await asyncio.sleep(60) # 429 오류는 1분간 대기
                        continue
                    else:
                        print(f"상세 정보 요청 실패: {detail_response.status} (ID: {movie_id}), 재시도 중... ({retry + 1}/{MAX_RETRIES})")
                        await asyncio.sleep(2 ** retry) # 지수 백오프: 1, 2, 4, 8초 대기
            except aiohttp.ClientError as e:
                print(f"API 요청 중 오류 발생: {e}, 재시도 중... ({retry + 1}/{MAX_RETRIES})")
                await asyncio.sleep(2 ** retry)
        
        # 모든 재시도가 실패했을 경우
        print(f"최종 실패: {movie_id} 영화 정보를 가져올 수 없습니다.")
        return None

async def fetch_movies_by_year(year):
    """
    비동기적으로 특정 연도의 모든 영화 데이터를 가져오는 함수
    """
    print(f"\n--- {year}년 영화 데이터를 수집 중입니다. ---")
    
    # discover/movie 엔드포인트를 위한 파라미터 설정
    params = {
        'language': 'ko-KR',
        'region': 'KR',
        'sort_by': 'popularity.desc',
        'primary_release_year': year,
        'page': 1,
        'vote_count.gte': 10,
        'include_adult': 'false', 
        'with_origin_country': 'KR', # 한국에서 제작한 영화만 포함 (이 필터를 다시 추가)
        'with_release_type': '2|3', # 제한적 개봉과 일반 개봉을 모두 포함
    }
    
    tasks = []
    
    async with aiohttp.ClientSession(headers=headers) as session:
        # 첫 페이지에서 전체 페이지 수 가져오기
        response = await session.get("https://api.themoviedb.org/3/discover/movie", params=params)
        data = await response.json()
        total_pages = data.get('total_pages', 1)

        # 각 페이지별로 영화 ID 가져오기
        for page in range(1, total_pages + 1):
            params['page'] = page
            response = await session.get("https://api.themoviedb.org/3/discover/movie", params=params)
            data = await response.json()
            movie_ids = [movie['id'] for movie in data.get('results', [])]
            
            # 각 영화 ID에 대한 상세 정보 요청 작업을 생성
            for movie_id in movie_ids:
                task = asyncio.create_task(fetch_movie_details(session, movie_id))
                tasks.append(task)
            print(f"  > {year}년 {page}/{total_pages} 페이지 영화 {len(movie_ids)}개 작업 등록 완료")
            
        # 모든 비동기 작업이 완료될 때까지 기다림
        results = await asyncio.gather(*tasks)
        
        # 유효한 데이터만 필터링하여 리스트에 추가
        for movie_data in results:
            if movie_data:
                all_movie_data.append(movie_data)

async def main():
    """
    메인 함수: 2005년부터 2024년까지 비동기 수집을 시작합니다.
    """
    start_time = time.time()
    await asyncio.gather(*(fetch_movies_by_year(year) for year in range(2005, 2025)))
    
    # 모든 데이터를 pandas DataFrame으로 변환
    df = pd.DataFrame(all_movie_data)
    
    # 'release_date' 열을 날짜/시간 형식으로 변환하여 올바르게 정렬
    df['release_date'] = pd.to_datetime(df['release_date'])
    # 'release_date'를 기준으로 오름차순 정렬
    df = df.sort_values(by='release_date')
    
    # DataFrame을 CSV 파일로 저장 (한글 깨짐 방지)
    df.to_csv("korean_movies_2005_2024_theatrical_full.csv", index=False, encoding='utf-8-sig')
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print("\n--- 모든 데이터 수집 및 저장 완료 ---")
    print(f"총 {len(df)}개 영화의 데이터가 'korean_movies_2005_2024_theatrical_full.csv' 파일에 저장되었습니다.")
    print(f"총 소요 시간: {elapsed_time:.2f}초")

if __name__ == "__main__":
    await main()



--- 2005년 영화 데이터를 수집 중입니다. ---

--- 2006년 영화 데이터를 수집 중입니다. ---

--- 2007년 영화 데이터를 수집 중입니다. ---

--- 2008년 영화 데이터를 수집 중입니다. ---

--- 2009년 영화 데이터를 수집 중입니다. ---

--- 2010년 영화 데이터를 수집 중입니다. ---

--- 2011년 영화 데이터를 수집 중입니다. ---

--- 2012년 영화 데이터를 수집 중입니다. ---

--- 2013년 영화 데이터를 수집 중입니다. ---

--- 2014년 영화 데이터를 수집 중입니다. ---

--- 2015년 영화 데이터를 수집 중입니다. ---

--- 2016년 영화 데이터를 수집 중입니다. ---

--- 2017년 영화 데이터를 수집 중입니다. ---

--- 2018년 영화 데이터를 수집 중입니다. ---

--- 2019년 영화 데이터를 수집 중입니다. ---

--- 2020년 영화 데이터를 수집 중입니다. ---

--- 2021년 영화 데이터를 수집 중입니다. ---

--- 2022년 영화 데이터를 수집 중입니다. ---

--- 2023년 영화 데이터를 수집 중입니다. ---

--- 2024년 영화 데이터를 수집 중입니다. ---
  > 2010년 1/3 페이지 영화 20개 작업 등록 완료
  > 2022년 1/3 페이지 영화 20개 작업 등록 완료
  > 2013년 1/4 페이지 영화 20개 작업 등록 완료
  > 2008년 1/3 페이지 영화 20개 작업 등록 완료
  > 2009년 1/3 페이지 영화 20개 작업 등록 완료
  > 2017년 1/5 페이지 영화 20개 작업 등록 완료
  > 2015년 1/4 페이지 영화 20개 작업 등록 완료
  > 2019년 1/5 페이지 영화 20개 작업 등록 완료
  > 2005년 1/3 페이지 영화 20개 작업 등록 완료
  > 2020년 1/4 페이지 영화 20개 작업 등록 완료
  > 2012년 1/3 페이지 영화